In [1]:
!pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import dlt, requests
print(f"dlt version is: {dlt.__version__}")

dlt version is: 1.12.3


In [2]:
@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [3]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant",
)

## Q2: How many rows were inserted into the zoomcamp_data collection?


In [5]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

In [ ]:
import json

with open("db.qdrant/meta.json") as f:
    meta = json.load(f)

print(json.dumps(meta, indent=2))